In [28]:
import os
from PIL import Image, ImageDraw, ImageFont

def concat_images_grid(image_folder, pos_value):
    image_files = [f for f in os.listdir(image_folder) if f.endswith('.png') and f.startswith(f"POS={pos_value}")]
    
    image_dict = {}
    for image_file in image_files:
        parts = image_file.split('_')
        dpos = int(parts[1].split('=')[1])
        drot = int(parts[2].split('=')[1].split('.')[0])
        image_dict[(dpos, drot)] = image_file
    
    dpos_values = sorted(set([k[0] for k in image_dict.keys()]))
    drot_values = sorted(set([k[1] for k in image_dict.keys()]))
    
    images_grid = []
    for dpos in dpos_values:
        row_images = []
        for drot in drot_values:
            image_path = os.path.join(image_folder, image_dict[(dpos, drot)])
            row_images.append(Image.open(image_path))
        images_grid.append(row_images)

    img_width, img_height = images_grid[0][0].size 
    total_width = img_width * len(drot_values)
    total_height = img_height * len(dpos_values)
    
    concatenated_image = Image.new('RGB', (total_width, total_height))
    
    draw = ImageDraw.Draw(concatenated_image)
    
    try:
        font = ImageFont.truetype("arial.ttf", 1000) 
    except IOError:
        font = ImageFont.load_default()

    for row_idx, row_images in enumerate(images_grid):
        for col_idx, image in enumerate(row_images):
            concatenated_image.paste(image, (col_idx * img_width, row_idx * img_height))
            
            label = f"DPOS={dpos_values[row_idx]}, DROT={drot_values[col_idx]}"
            text_position = (col_idx * img_width + 10, row_idx * img_height + 10)
            draw.text(text_position, label, font=font, fill="white")
    
    output_folder = 'hpsearch_concat'
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    output_filename = os.path.join(output_folder, f"POS={pos_value}_concatenated_grid_labeled.png")
    concatenated_image.save(output_filename)
    print(f"Concatenated image with labels saved as {output_filename}")

concat_images_grid('hpsearch', pos_value=1)


Concatenated image with labels saved as hpsearch_concat/POS=1_concatenated_grid_labeled.png
